<a href="https://colab.research.google.com/github/hrishavranjan/Python-Basic-Collab-Codes/blob/main/LAB_5_AI_17_01_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import heapq

class PuzzleState:
    def _init_(self, board, goal, g=0, h=0):
        self.board, self.goal, self.g, self.h = board, goal, g, h
        self.f, self.empty_pos = g + h, next((i, j) for i, row in enumerate(board) for j, val in enumerate(row) if val == 0)

    def _lt_(self, other): return self.f < other.f
    def is_goal(self): return self.board == self.goal
    def neighbors(self):
        x, y = self.empty_pos
        moves = [(x+dx, y+dy) for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)] if 0 <= x+dx < 3 and 0 <= y+dy < 3]
        return [self._move(nx, ny) for nx, ny in moves]

    def _move(self, nx, ny):
        new_board = [row[:] for row in self.board]
        x, y = self.empty_pos
        new_board[x][y], new_board[nx][ny] = new_board[nx][ny], new_board[x][y]
        return PuzzleState(new_board, self.goal, self.g + 1)


def h_misplaced(state):
    return sum(1 for i, row in enumerate(state.board) for j, val in enumerate(row) if val and val != state.goal[i][j])

def h_manhattan(state):
    return sum(abs(i - (val - 1) // 3) + abs(j - (val - 1) % 3) for i, row in enumerate(state.board) for j, val in enumerate(row) if val)

def a_star(start, goal, heuristic):
    start_state = PuzzleState(start, goal, h=heuristic(PuzzleState(start, goal)))
    open_list, closed_set, nodes = [start_state], set(), 0
    while open_list:
        state = heapq.heappop(open_list)
        nodes += 1
        if state.is_goal():
            return nodes, state.g
        closed_set.add(tuple(map(tuple, state.board)))
        for neighbor in state.neighbors():
            if tuple(map(tuple, neighbor.board)) not in closed_set:
                neighbor.h = heuristic(neighbor)
                neighbor.f = neighbor.g + neighbor.h
                heapq.heappush(open_list, neighbor)
    return -1, -1

if _name_ == "_main_":
    start, goal = [[1, 2, 3], [4, 0, 5], [7, 8, 6]], [[1, 2, 3], [4, 5, 6], [7, 8, 0]]
    for h, name in [(h_misplaced, "H1 (Misplaced Tiles)"), (h_manhattan, "H2 (Manhattan Distance)")]:
        nodes, depth = a_star(start, goal, h)
        print(f"Using {name}: Nodes explored: {nodes}, Solution depth: {depth}")